<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Modificado en 2018-1 y 2018-2 por Equipo Docente IIC2233. Creado por equipo docente en 2015-1.</font>
</p>

# Serialización de objetos

Como revisamos en el material anterior, toda la información que almacena un computador se guarda en base a _bits_ (esto es, ceros y unos). Por ejemplo, imaginemos que buscamos guardar una estructura de datos o una instancia de una clase. De alguna forma, estos datos —que probablemente no tienen una estructura lineal— deben ser guardados como una serie o secuencia de _bytes_. Aquí es cuando aparece el término de **serialización**.

Este concepto se refiere al procedimiento de transformar cualquier objeto en una secuencia o serie de _bytes_. Esto nos permite almacenar el estado de un objeto de forma persistente, por ejemplo en un archivo o una base de datos que podemos consultar más tarde. También nos permite enviar el objeto a otros computadores y programas.

## _Pickle_

La librería _pickle_ de Python permite guardar y cargar casi cualquier objeto de Python, incluyendo listas. Esta librería ofrece los métodos:
- `dumps`: serializa un objeto, es decir, lo guarda.
- `loads`: deserializa un objeto serializado, es decir, carga un objeto a su estado original.

Una vez que un objeto es serializado, este es persistente y está listo para volver a ser usado en el futuro por el mismo programa.

In [3]:
import pickle

tupla = ("a", 1, 3, "hola")
serializacion = pickle.dumps(tupla)

print(serializacion)
print(type(serializacion))
print(pickle.loads(serializacion))
print(type(pickle.loads(serializacion)))

b'\x80\x03(X\x01\x00\x00\x00aq\x00K\x01K\x03X\x04\x00\x00\x00holaq\x01tq\x02.'
<class 'bytes'>
('a', 1, 3, 'hola')
<class 'tuple'>


_Pickle_ también nos ofrece los métodos `dump` y `load` (casi el mismo nombre que antes pero sin la _s_). Estos métodos también serializan y deserializan, pero a través de archivos: 

- `dump`: guarda un archivo con el objeto serializado.
- `load`: deserializa un objeto almacenado en un archivo (lo trae de vuelta).

In [4]:
lista = [1, 2, 3, 7, 8, 3]

with open("mi_lista.bin", 'wb') as file:
    pickle.dump(lista, file)

with open("mi_lista.bin", 'rb') as file:
    lista_cargada = pickle.load(file)

print(f"¿Las listas son iguales? {lista == lista_cargada}")
print(f"¿Las listas son el mismo objeto? {lista is lista_cargada}")

¿Las listas son iguales? True
¿Las listas son el mismo objeto? False


_Pickle_ es un módulo no seguro. Esto implica que **nunca** debes cargar un archivo _pickle_ cuando no conoces su procedencia, ya que éste podría ejecutar código malicioso en tu computador. No entraremos en detalles sobre cómo inyectar código a través del módulo _pickle_, pero los interesados pueden revisar este [enlace](http://www.cs.jhu.edu/~s/musings/pickle.html).


### Personalizar la serialización

Cuando _pickle_ trata de serializar un objeto, lo primero que hará es verificar que el objeto que se quiere serializar sea de una clase que tenga implementado el método `__getstate__`. Este método debe retornar un diccionario con los atributos que se quieren serializar. Si `__getstate__` no estuviese implementado, entonces _pickle_ guardará el atributo `__dict__` del objeto. Recuerda que el atributo `__dict__` es un diccionario que guarda todos los atributos y métodos de un objeto. Por ejemplo, `o.atributo` es equivalente a `o.__dict__["atributo"]` y `o.atributo = 42` es equivalente a `o.__dict__["atributo"] = 42`.

El método `__getstate__` nos permite personalizar la serialización del objeto. Usando este método podemos crear un diccionario que contenga solo la información que deseamos guardar. 

In [5]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"
        
    def __getstate__(self):
        """
        Retorna el estado actual del objeto, para que sea serializado por pickle
        
        Aquí creamos una copia del diccionario actual, para modificar la copia 
        y no el objeto original
        """
        nueva = self.__dict__.copy()
        nueva.update({"mensaje" : "¡Me están serializando!"})
        return nueva

In [6]:
original = Persona("Juan", 30)
print("Mensaje original:", original.mensaje)
serializado = pickle.dumps(original)
deserializado = pickle.loads(serializado)

# El objeto original sigue igual
print("Mensaje original:", original.mensaje)
print("Mensaje deserializado:", deserializado.mensaje)

Mensaje original: No pasa nada
Mensaje original: No pasa nada
Mensaje deserializado: ¡Me están serializando!


### Personalizar la deserialización

De forma análoga, podemos personalizar la **deserialización**. Para esto debemos implementar el método `__setstate__`, que se ejecutará cada vez que llamemos a `load` o `loads`. El método `__setstate__` recibe como argumento el diccionario que representa el estado del objeto que fue serializado. Luego debe asignarlo al diccionario del objeto `self.__dict__ = diccionario_con_estado`. Esto no impide que se realicen otras acciones que modifiquen `diccionario_con_estado` antes o después de la asignación.

Si el método `__setstate__` no estuviese implementado, entonces se asignará al `__dict__` del objeto el estado deserializado sin realizar otras acciones adicionales.

In [9]:
class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"
        
    def __getstate__(self):
        nueva = self.__dict__.copy()
        nueva.update({"mensaje": "¡Me están serializando!"})
        # esto es lo que será serializado por pickle
        return nueva 

    def __setstate__(self, state):
        print("Objeto recién deserializado, actualizando su estado")
        state.update({"nombre": state["nombre"] + " deserializado"})
        self.__dict__ = state

In [10]:
original = Persona("Juan", 30)
print("Nombre original:", original.nombre)
serializado = pickle.dumps(original)

Nombre original: Juan


In [13]:
des = pickle.loads(serializado)
des.nombre

Objeto recién deserializado, actualizando su estado


'Juan deserializado'

Una aplicación práctica de los métodos `__getstate__` y `__setstate__` es cuando necesitamos serializar un objeto que contiene un atributo que depende de las condiciones actuales del programa. Por ejemplo, imaginemos que un objeto mantiene una conexión a una base de datos. Supongamos, además, que esta conexión tiene atributos relacionados con el estado del programa: tiempo de conexión, puertos utilizados, _host_, entre otros. Cuando guardamos el objeto, deberíamos eliminar la conexión, ya que este atributo no podrá ser utilizado en otra instancia del programa. Para lograr esto usamos el método `__getstate__`. 

Cuando se cargue el mismo objeto desde el archivo serializado, será necesario volver a crear la conexión con las condiciones del programa. Para realizar esto tendremos que implementar `__setstate__`.

## Serialización de objetos con JSON

Una de las desventajas de los objetos serializados con `pickle` es que sólo pueden ser deserializados por otros programas escritos en Python. Por otra parte, _**JSON**_ (JavaScript Object Notation) es un formato estándar de intercambio de datos que puede ser interpretado por muchos lenguajes. _JSON_ además es _human-readable_, es decir, puede ser fácilmente leído y entendido por humanos.

**El formato en que almacena la información es similar a los diccionarios de Python.**

En _JSON_ sólo es posible serializar instancias de `int`, `str`, `float`, `dict`, `bool`, `list`, `tuple` y `NoneType`, de acuerdo a esta tabla de transformación que puedes revisar en [este link](https://docs.python.org/3.6/library/json.html#encoders-and-decoders). Sin embargo, por defecto no es posible serializar funciones o instancias de otras clases.

En Python, existe un módulo llamado `json` que provee métodos para serializar objetos en el  formato _JSON_. Este módulo provee una interfaz similar a la de `pickle`, es decir, los métodos `dump`(`s`) y `load`(`s`). 

In [10]:
from itertools import count
import json

class Persona:
    id = count()
    
    def __init__(self, nombre, edad, estado_civil):
        self.nombre = nombre
        self.edad = edad
        self.estado_civil = estado_civil
        self.id_ = next(self.id)

In [11]:
p = Persona("Juan", 35, "Soltero")

json_string = json.dumps(p.__dict__)
print("Datos en formato JSON:", type(json_string), json_string)

json_deserializado = json.loads(json_string)
print("Datos en formato Python:", type(json_deserializado), json_deserializado)

Datos en formato JSON: <class 'str'> {"nombre": "Juan", "edad": 35, "estado_civil": "Soltero", "id_": 0}
Datos en formato Python: <class 'dict'> {'nombre': 'Juan', 'edad': 35, 'estado_civil': 'Soltero', 'id_': 0}


### Personalizar la serialización

Cuando queremos guardar un objeto como _JSON_ podemos personalizar la transformación utilizando un `json.JSONEncoder`, de forma análoga a como lo hicimos `__getstate__`. Para esto debemos crear una clase que hereda de la clase `json.JSONEncoder` y sobreescribir el método `default`:

In [10]:
from datetime import datetime


class PersonaEncoder(json.JSONEncoder):
       def default(self, obj):
            """Serializa en forma personalizada el tipo de objeto Persona"""
            
            if isinstance(obj, Persona):
                return {'Persona_id': obj.id_, 
                        'nombre': obj.nombre, 
                        'edad': obj.edad, 
                        'estado_civil': obj.estado_civil, 
                        'ano_nacimiento' : datetime.now().year - obj.edad}
            
            # Mantenemos la serialización por defecto para otros tipos
            return super().default(obj)

        
# Creamos tres instancias
p1 = Persona("Juanita", 37, "Soltera")
p2 = Persona("Jorge", 33, "Casado")
p3 = Persona("Mariela", 24, "Soltera")

Probamos la serialización usando el _default_:

In [11]:
json_string = json.dumps(p1.__dict__)

json_string

'{"nombre": "Juanita", "edad": 37, "estado_civil": "Soltera", "id_": 1}'

Ahora, serializamos usando el método personalizado:

In [12]:
json_string = json.dumps(p1, cls=PersonaEncoder)
print(json_string)

json_string = json.dumps(p2, cls=PersonaEncoder)
print(json_string)

json_string = json.dumps(p3, cls=PersonaEncoder)
print(json_string)

{"Persona_id": 1, "nombre": "Juanita", "edad": 37, "estado_civil": "Soltera", "ano_nacimiento": 1981}
{"Persona_id": 2, "nombre": "Jorge", "edad": 33, "estado_civil": "Casado", "ano_nacimiento": 1985}
{"Persona_id": 3, "nombre": "Mariela", "edad": 24, "estado_civil": "Soltera", "ano_nacimiento": 1994}


### Personalizar la deserialización

Cuando queremos transformar un _JSON_ a un objeto Python podemos utilizar los `object_hook`, de forma análoga a como lo hicimos con `__setstate__`. 

El `object_hook` es un parámetro de los métodos `load` y `loads`, que espera una función que sea capaz de manejar un diccionario y retorne un objeto de Python. En el proceso de deserialización, todo objeto _JSON_ será convertido a un diccionario de Python, y luego pasado a la función `object_hook` para hacer la transformación final.

Por ejemplo, si queremos cargar datos _JSON_ en una lista de tuplas en vez de un diccionario:

In [13]:
def funcion_hook(diccionario):
    return [(key, value) for key, value in diccionario.items()]

json_string = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado", "puntaje": 90.5}'
datos = json.loads(json_string, object_hook=funcion_hook)

datos

[('nombre', 'Jorge'),
 ('edad', 34),
 ('estado_civil', 'casado'),
 ('puntaje', 90.5)]

Es importante recordar que **todo** objeto de _JSON_ es convertido a un diccionario, y luego entregado al `object_hook`. Esto tiene implicancias cuando tenemos un _JSON_ con objetos anidados:

In [14]:
def funcion_hook(diccionario):
    """Esta función transforma un diccionario en un número"""
    print(f"Me llegó el diccionario: {diccionario}")
    valor = 33 if len(diccionario) > 1 else 42
    print(f"Lo transformaré en {valor}\n")
    return valor


json_string = '{"a": {"b": 1, "c": [2, 3, {}], "d": null, "e": {"f": true}}}'
datos = json.loads(json_string, object_hook=funcion_hook)

datos

Me llegó el diccionario: {}
Lo transformaré en 42

Me llegó el diccionario: {'f': True}
Lo transformaré en 42

Me llegó el diccionario: {'b': 1, 'c': [2, 3, 42], 'd': None, 'e': 42}
Lo transformaré en 33

Me llegó el diccionario: {'a': 33}
Lo transformaré en 42



42

¿Qué pasó ahí arriba? Lo primero que podemos notar es que el proceso de conversión funciona de adentro hacia afuera, y además que el resultado que entregue `object_hook` se arrastra en el proceso.